In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split , KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import math
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [2]:


# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
Nasa=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\cocomonasa_2_dataset.csv",header=None,index_col=0)
columns_Nasa=['ProjName','CatofApp','ForG','Center','YearofDev','DevMode','rely','data','cplx','time','stor','virt','turn','acap','aexp','pcap','vexp','lexp','modp','tool','sced','loc','Effort']
Nasa.set_axis(columns_Nasa,axis='columns',inplace=True)
Nasa.rename_axis("Features", axis=1,inplace=True)
Nasa.rename_axis("Projects ID", axis=0,inplace=True)
Nasa.reset_index(inplace=True)
Nasa.drop(columns=['Projects ID'], inplace=True)

#Nasa dataset preprocessing
def NASA_quantifier(x):
    if x=='vl':
        return 0
    elif x=='l':
        return 1
    elif x=='n':
        return 2
    elif x=='h':
        return 3
    elif x=='vh':
        return 4
    elif x=='xh':
        return 5
    else:
        return x

Nasa=Nasa.applymap(NASA_quantifier)
dataset = Nasa

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort'])  # Features
y = dataset['Effort'].values  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Step 2: Preprocess Data
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

# Step 3: Feature Selection using Harmony Search
def objective_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    
    X_selected = X.iloc[:, selected_features]
    X_selected = pd.get_dummies(X_selected)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=1)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
    
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)

#==================================================================================================================
# Harmony Search Parameters
num_features = X.shape[1]
hms = 20  # Harmony memory size
iterations = 100  # Number of iterations

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < 0.5:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj > current_obj:
        harmony_memory[rnd_choice] = new_harmony

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print(selected_features)
# Step 4: Train SVR Model using selected features
#selected_X = X_scaled[:, np.array(selected_features) == 1]
#=============================================================================================================               
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    
X_selected = X.iloc[:, selected_features]
X_selected = pd.get_dummies(X_selected)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_RMSE=[]
mean_MMRE=[]
mean_MAE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Step 3: Build the ANN model.
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
])

    # Step 4: Compile the model.
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Step 5: Train the model.
    model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=1)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")
print(selected_features)


Epoch 1/5
10/10 [==============================] - 0s 1ms/step - loss: 2039652.5000
Epoch 2/5
10/10 [==============================] - 0s 2ms/step - loss: 2038945.5000
Epoch 3/5
10/10 [==============================] - 0s 2ms/step - loss: 2038298.1250
Epoch 4/5
10/10 [==============================] - 0s 2ms/step - loss: 2037651.2500
Epoch 5/5
1/1 [==============================] - 0s 70ms/step
Epoch 1/5
10/10 [==============================] - 0s 1ms/step - loss: 1960171.0000
Epoch 2/5
10/10 [==============================] - 0s 0s/step - loss: 1959469.0000
Epoch 3/5
10/10 [==============================] - 0s 0s/step - loss: 1958785.2500
Epoch 4/5
10/10 [==============================] - 0s 2ms/step - loss: 1957898.7500
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
10/10 [==============================] - 0s 1ms/step - loss: 1022614.6875
Epoch 2/5
10/10 [==============================] - 0s 0s/step - loss: 1022180.6250
Epoch 3/5
10/10 [======================

10/10 [==============================] - 0s 2ms/step - loss: 1953004.1250
Epoch 4/5
10/10 [==============================] - 0s 1ms/step - loss: 1952234.0000
Epoch 5/5
1/1 [==============================] - 0s 29ms/step
Epoch 1/5
10/10 [==============================] - 0s 1ms/step - loss: 2036792.3750
Epoch 2/5
10/10 [==============================] - 0s 0s/step - loss: 2035930.0000
Epoch 3/5
10/10 [==============================] - 0s 0s/step - loss: 2035047.6250
Epoch 4/5
10/10 [==============================] - 0s 2ms/step - loss: 2034079.6250
Epoch 5/5
1/1 [==============================] - 0s 29ms/step
Epoch 1/5
10/10 [==============================] - 0s 523us/step - loss: 1961334.0000
Epoch 2/5
10/10 [==============================] - 0s 0s/step - loss: 1960840.8750
Epoch 3/5
10/10 [==============================] - 0s 0s/step - loss: 1960376.0000
Epoch 4/5
10/10 [==============================] - 0s 2ms/step - loss: 1959892.3750
Epoch 5/5
1/1 [==============================] -

10/10 [==============================] - 0s 1ms/step - loss: 1959866.3750
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
10/10 [==============================] - 0s 2ms/step - loss: 1022023.6875
Epoch 2/5
10/10 [==============================] - 0s 0s/step - loss: 1021448.5625
Epoch 3/5
10/10 [==============================] - 0s 387us/step - loss: 1020958.1875
Epoch 4/5
10/10 [==============================] - 0s 1ms/step - loss: 1020459.2500
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
10/10 [==============================] - 0s 1ms/step - loss: 1353313.8750
Epoch 2/5
10/10 [==============================] - 0s 667us/step - loss: 1353093.3750
Epoch 3/5
10/10 [==============================] - 0s 585us/step - loss: 1352890.3750
Epoch 4/5
10/10 [==============================] - 0s 112us/step - loss: 1352646.2500
Epoch 5/5
1/1 [==============================] - 0s 45ms/step
Epoch 1/5
10/10 [==============================] - 0s 3ms/ste

1/1 [==============================] - 0s 42ms/step
Epoch 1/5
10/10 [==============================] - 0s 623us/step - loss: 1955447.5000
Epoch 2/5
10/10 [==============================] - 0s 0s/step - loss: 1955037.6250
Epoch 3/5
10/10 [==============================] - 0s 0s/step - loss: 1954630.6250
Epoch 4/5
10/10 [==============================] - 0s 2ms/step - loss: 1954241.1250
Epoch 5/5
1/1 [==============================] - 0s 41ms/step
Epoch 1/5
10/10 [==============================] - 0s 2ms/step - loss: 2038252.3750
Epoch 2/5
10/10 [==============================] - 0s 0s/step - loss: 2037389.3750
Epoch 3/5
10/10 [==============================] - 0s 0s/step - loss: 2036519.6250
Epoch 4/5
10/10 [==============================] - 0s 2ms/step - loss: 2035488.3750
Epoch 5/5
1/1 [==============================] - 0s 37ms/step
Epoch 1/5
10/10 [==============================] - 0s 749us/step - loss: 1960546.7500
Epoch 2/5
10/10 [==============================] - 0s 0s/step - loss

1/1 [==============================] - 0s 36ms/step
Epoch 1/5
10/10 [==============================] - 0s 888us/step - loss: 1960682.0000
Epoch 2/5
10/10 [==============================] - 0s 778us/step - loss: 1960235.6250
Epoch 3/5
10/10 [==============================] - 0s 889us/step - loss: 1959808.6250
Epoch 4/5
10/10 [==============================] - 0s 1ms/step - loss: 1959298.0000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
10/10 [==============================] - 0s 891us/step - loss: 1022345.0000
Epoch 2/5
10/10 [==============================] - 0s 815us/step - loss: 1022024.1875
Epoch 3/5
10/10 [==============================] - 0s 777us/step - loss: 1021732.5625
Epoch 4/5
10/10 [==============================] - 0s 889us/step - loss: 1021409.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
10/10 [==============================] - 0s 891us/step - loss: 1352076.2500
Epoch 2/5
10/10 [==============================] - 0

...

10/10 [==============================] - 0s 835us/step - loss: 1022222.2500
Epoch 4/5
10/10 [==============================] - 0s 816us/step - loss: 1021983.8750
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
10/10 [==============================] - 0s 779us/step - loss: 1353726.8750
Epoch 2/5
10/10 [==============================] - 0s 792us/step - loss: 1353175.8750
Epoch 3/5
10/10 [==============================] - 0s 908us/step - loss: 1352722.5000
Epoch 4/5
10/10 [==============================] - 0s 779us/step - loss: 1352203.8750
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
10/10 [==============================] - 0s 889us/step - loss: 1954182.8750
Epoch 2/5
10/10 [==============================] - 0s 778us/step - loss: 1953527.0000
Epoch 3/5
10/10 [==============================] - 0s 777us/step - loss: 1952719.6250
Epoch 4/5
10/10 [==============================] - 0s 787us/step - loss: 1951910.8750
Epoch 5/5
1/1 [===========

10/10 [==============================] - 0s 778us/step - loss: 1952666.6250
Epoch 4/5
10/10 [==============================] - 0s 779us/step - loss: 1951969.8750
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
10/10 [==============================] - 0s 836us/step - loss: 2041256.6250
Epoch 2/5
10/10 [==============================] - 0s 821us/step - loss: 2040793.2500
Epoch 3/5
10/10 [==============================] - 0s 834us/step - loss: 2040434.1250
Epoch 4/5
10/10 [==============================] - 0s 1ms/step - loss: 2040092.7500
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
10/10 [==============================] - 0s 890us/step - loss: 1961670.0000
Epoch 2/5
10/10 [==============================] - 0s 814us/step - loss: 1961336.0000
Epoch 3/5
10/10 [==============================] - 0s 779us/step - loss: 1961049.7500
Epoch 4/5
10/10 [==============================] - 0s 781us/step - loss: 1960683.5000
Epoch 5/5
1/1 [=============

10/10 [==============================] - 0s 834us/step - loss: 1962248.6250
Epoch 4/5
10/10 [==============================] - 0s 1ms/step - loss: 1961469.8750
Epoch 5/5
1/1 [==============================] - 0s 38ms/step
Epoch 1/5
10/10 [==============================] - 0s 835us/step - loss: 1023106.2500
Epoch 2/5
10/10 [==============================] - 0s 1ms/step - loss: 1022660.3125
Epoch 3/5
10/10 [==============================] - 0s 913us/step - loss: 1022296.0000
Epoch 4/5
10/10 [==============================] - 0s 778us/step - loss: 1021958.1875
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
10/10 [==============================] - 0s 1ms/step - loss: 1351953.1250
Epoch 2/5
10/10 [==============================] - 0s 776us/step - loss: 1351536.0000
Epoch 3/5
10/10 [==============================] - 0s 889us/step - loss: 1351123.5000
Epoch 4/5
10/10 [==============================] - 0s 777us/step - loss: 1350610.3750
Epoch 5/5
1/1 [=================

10/10 [==============================] - 0s 835us/step - loss: 1352618.7500
Epoch 4/5
10/10 [==============================] - 0s 716us/step - loss: 1352368.6250
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
10/10 [==============================] - 0s 797us/step - loss: 1956014.7500
Epoch 2/5
10/10 [==============================] - 0s 808us/step - loss: 1955672.3750
Epoch 3/5
10/10 [==============================] - 0s 724us/step - loss: 1955364.8750
Epoch 4/5
10/10 [==============================] - 0s 781us/step - loss: 1955089.5000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
10/10 [==============================] - 0s 834us/step - loss: 2041220.1250
Epoch 2/5
10/10 [==============================] - 0s 777us/step - loss: 2040204.7500
Epoch 3/5
10/10 [==============================] - 0s 667us/step - loss: 2039297.2500
Epoch 4/5
10/10 [==============================] - 0s 778us/step - loss: 2038420.7500
Epoch 5/5
1/1 [===========

10/10 [==============================] - 0s 778us/step - loss: 2039145.5000
Epoch 4/5
10/10 [==============================] - 0s 667us/step - loss: 2038467.6250
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
10/10 [==============================] - 0s 890us/step - loss: 1959776.8750
Epoch 2/5
10/10 [==============================] - 0s 778us/step - loss: 1959126.0000
Epoch 3/5
10/10 [==============================] - 0s 667us/step - loss: 1958524.5000
Epoch 4/5
10/10 [==============================] - 0s 667us/step - loss: 1957789.2500
Epoch 5/5
1/1 [==============================] - 0s 31ms/step
Epoch 1/5
10/10 [==============================] - 0s 889us/step - loss: 1022276.1875
Epoch 2/5
10/10 [==============================] - 0s 666us/step - loss: 1021790.0625
Epoch 3/5
10/10 [==============================] - 0s 777us/step - loss: 1021353.3125
Epoch 4/5
10/10 [==============================] - 0s 666us/step - loss: 1020948.5625
Epoch 5/5
1/1 [===========